In [13]:
# imports and dependencies
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, date, time, timezone
import yfinance as yf
from textblob import TextBlob

## EXTRACT YAHOO FINANCE PRICES 

In [7]:
# Load PLTR stock closing prices from YFinance
pltr_yf = yf.download('PLTR',
                       start='2020-01-01',
                       end='2021-05-31',
                       progress=False,
                       infer_datetime_format= True,
                       parse_dates = True,
                       interval='1d'

)
# Create DataFrame
pltr_yf_df = pd.DataFrame(pltr_yf)
# Reorder DataFrame columns
pltr_yf_df=pltr_yf_df.sort_values('Date')
pltr_yf_df.drop(columns=['Open','High','Low','Volume','Adj Close'],inplace=True)
pltr_yf_df.info()
pltr_yf_df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 137 entries, 2020-09-30 to 2021-04-16
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   137 non-null    float64
dtypes: float64(1)
memory usage: 2.1 KB


,Close
Date,
2020-09-30,9.500000
2020-10-01,9.460000
2020-10-02,9.200000
2020-10-05,9.030000
2020-10-06,9.900000
...,...
2021-04-12,23.340000
2021-04-13,25.420000
2021-04-14,23.700001


## EXTRACT TWITTER DATA 

In [8]:
# Set and read PLTR Twitter data
pltr_twitter_df = pd.read_csv('pltr_twitter_df.csv', parse_dates=True, infer_datetime_format=True,low_memory=False)
pltr_twitter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441537 entries, 0 to 441536
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Date    441536 non-null  object
 1   ID      441537 non-null  object
 2   Tweet   441536 non-null  object
dtypes: object(3)
memory usage: 10.1+ MB


In [11]:
# Clean, sort and view the data
pltr_twitter_df.drop_duplicates()
pltr_twitter_df=pltr_twitter_df.sort_values('Date')
pltr_twitter_df.head()

,Date,ID,Tweet
304526,2020-09-29 00:10:04 UTC,1310733546969952256,Don’t forget buy $PLTR tomorrow. Don’t sleep
304525,2020-09-29 00:48:23 UTC,1310743188496429058,Why? Why do they have to keep giving us new in...
304524,2020-09-29 00:53:35 UTC,1310744498633351168,Got $PLTR on my radar tomorrow. It's a growing...
304523,2020-09-29 01:16:41 UTC,1310750312328699905,The data-mining-software company Palantir Tech...
304522,2020-09-29 01:32:08 UTC,1310754200381423618,What you need to know about about Palantir lis...


## CREATE FOR LOOP AND TB ANALYSIS 

In [ ]:
#Create For loop to iterate through tweets and conduct TextBlob analysis of PLTR Twitter df
from textblob import TextBlob
pltr_tb_sentiments = []
for index, row in pltr_twitter_df.iterrows():
    try:
        text = row["Tweet"]
        date = row["Date"]
        analysis = TextBlob(text).sentiment
        analysisSub= TextBlob(text).subjectivity
        analysisPol= TextBlob(text).polarity
        pltr_tb_sentiments.append({
            "text": text,
            "date": date,
            "TB Subjectivity Score": analysisSub,
            "TB Polarity Score":analysisPol
        })
    except AttributeError:
        pass
pltr_tb_sentiment_df = pd.DataFrame(pltr_tb_sentiments)
pltr_tb_sentiment_df

In [ ]:
#Drop duplicates
pltr_tb_sentiment_df.drop_duplicates(inplace=True)
pltr_tb_sentiment_df
#Create & Save PLTR TEXT BLOB Sentiment as CSV file
pltr_tb_sentiment_df.to_csv("ptlr_tb_sentiment_df.csv", index=True)

In [ ]:
### CALCULATE AVERAGE BY DATE
twitter_tb_sent_avg = pltr_tb_sentiment_df.groupby('date', as_index=True, sort=True)['TB Subjectivity Score','TB Subjectivity Score'].mean()
twitter_tb_sent_avg.info()
twitter_tb_sent_avg

In [ ]:
pltr_yf_tb_twitter = pd.concat([pltr_yf_df,twitter_tb_sent_avg],axis=1,join='outer')
# pltr_yf_twitter.dropna(inplace=True)
pltr_yf_twitter.dropna(inplace=True)
pltr_yf_twitter.head()

## Open NewApi CSV

In [18]:
# Read NewswAPI 
pltr_newsapi_df = pd.read_csv('Newsapi.csv')

pltr_newsapi_df.head()

,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content
0,0,"{'id': None, 'name': 'Benzinga'}",Tyler Bundy,Palantir Stock Trades Higher: A Technical Anal...,Palantir Technology Inc. (NYSE:PLTR) shares we...,https://www.benzinga.com/trading-ideas/long-id...,https://cdn2.benzinga.com/files/imagecache/og_...,2021-03-25T19:06:00Z,Palantir Technology Inc. (NYSE:PLTR) shares we...
1,1,"{'id': 'business-insider', 'name': 'Business I...",Emily Graffeo,Palantir jumps after securing 5-year contract ...,Summary List PlacementShares of Palantir gaine...,https://www.businessinsider.com/palantir-stock...,https://i.insider.com/605213680347cb00184b63a3...,2021-04-05T13:16:57Z,Shares of Palantir gained 3.47% early Monday a...
2,2,"{'id': 'reuters', 'name': 'Reuters'}",Reuters Staff,Former Credit Suisse exec backed SPAC to take ...,Sarcos Robotics Inc said on Tuesday it will go...,https://www.reuters.com/article/us-sarcos-m-a-...,https://s1.reutersmedia.net/resources_v2/image...,2021-04-06T10:53:00Z,By Reuters Staff\r\n(Reuters) - Sarcos Robotic...
3,3,"{'id': None, 'name': 'Slashdot.org'}",BeauHD,Coinbase Sets Direct Listing For April 14,"Coinbase, the largest U.S. cryptocurrency exch...",https://slashdot.org/story/21/04/01/2124242/co...,https://a.fsdn.com/sd/topics/bitcoin_64.png,2021-04-02T01:25:00Z,The company's registration statement for the l...
4,4,"{'id': 'reuters', 'name': 'Reuters'}",Reuters Staff,Breakingviews - Capital Calls - Cellebrite goo...,SOMETHING TO CELLEBRITE. Controversy over cops...,https://www.reuters.com/article/us-global-fina...,https://static.reuters.com/resources/r/?m=02&d...,2021-04-08T19:41:00Z,By Reuters Staff\r\nA Cellebrite forensic devi...


## CREATE FOR LOOP AND TB ANALYSIS

In [17]:
#Create For loop to iterate through tweets and conduct TextBlob analysis of PLTR Twitter df

pltr_tb_newsapi_sentiments = []
for index, row in pltr_newsapi_df.iterrows():
    try:
        text = row["content"]
        date = row["publishedAt"]
        analysis = TextBlob(text).sentiment
        analysisSub= TextBlob(text).subjectivity
        analysisPol= TextBlob(text).polarity
        pltr_tb_sentiments.append({
            "text": text,
            "date": date,
            "TB Subjectivity Score": analysisSub,
            "TB Polarity Score":analysisPol
        })
    except AttributeError:
        pass
pltr_tb_newsapi_sentiment_df = pd.DataFrame(pltr_tb_newsapi_sentiments)
pltr_tb_newsapi_sentiment_df

""
